## Практическая работа №4
##### Сиразетдинов Рустем (Вариант - 20)

1.0 Подготовка датасета для последующего решения задачи

In [75]:
import pandas
import numpy as np
import io

data = pandas.read_csv('C:\\Users\\averu\\Documents\\git_local\\programming-practice\\IDA-practice-4\\BankChurners.csv', index_col='CLIENTNUM')

data_set = data.loc[:, data.columns.isin(['Marital_Status', 'Credit_Limit', 'Customer_Age', 'Gender', 'Education_Level', 'Income_Category', 'Months_on_book'])]

data_set = data_set.dropna()

data_set['Gender']          = np.where(data_set['Gender'] == 'M', 1, 0)

data_set['Education_Level'] = np.where(data_set['Education_Level'] == 'Unknown' , 1 , data_set['Education_Level'])
data_set['Education_Level'] = np.where(data_set['Education_Level'] == 'Uneducated' , 1 , data_set['Education_Level'])
data_set['Education_Level'] = np.where(data_set['Education_Level'] == 'High School' , 2 , data_set['Education_Level'])
data_set['Education_Level'] = np.where(data_set['Education_Level'] == 'Graduate' , 3 , data_set['Education_Level'])
data_set['Education_Level'] = np.where(data_set['Education_Level'] == 'College', 3 , data_set['Education_Level'])
data_set['Education_Level'] = np.where(data_set['Education_Level'] == 'Post-Graduate' , 4 , data_set['Education_Level'])
data_set['Education_Level'] = np.where(data_set['Education_Level'] == 'Doctorate' , 4 , data_set['Education_Level'])

data_set['Income_Category'] = np.where(data_set['Income_Category'] == 'Unknown' , 0, data_set['Income_Category'])
data_set['Income_Category'] = np.where(data_set['Income_Category'] == 'Less than $40K' , 1, data_set['Income_Category'])
data_set['Income_Category'] = np.where(data_set['Income_Category'] == '$40K - $60K' , 2, data_set['Income_Category'])
data_set['Income_Category'] = np.where(data_set['Income_Category'] == '$60K - $80K' , 3, data_set['Income_Category'])
data_set['Income_Category'] = np.where(data_set['Income_Category'] == '$80K - $120K' , 4, data_set['Income_Category'])
data_set['Income_Category'] = np.where(data_set['Income_Category'] == '$120K +' , 5, data_set['Income_Category'])

data_set['Credit_Limit']    = np.where([ (float(x) > 10000) for x in data_set['Credit_Limit'] ] , 1 , 0)
# data_set['Credit_Limit']    = [ (float(x) > 10000) for x in data_set['Credit_Limit'] ]

data_set['Marital_Status'] = np.where(data_set['Marital_Status'] == 'Married' , False, data_set['Marital_Status'])
data_set['Marital_Status'] = np.where(data_set['Marital_Status'] == 'Single' , True, data_set['Marital_Status'])
data_set['Marital_Status'] = np.where(data_set['Marital_Status'] == 'Divorced' , True, data_set['Marital_Status'])
data_set['Marital_Status'] = np.where(data_set['Marital_Status'] == 'Unknown' , True, data_set['Marital_Status'])

M_status = data_set.loc[:, data_set.columns.isin(['Marital_Status'])]
# settings = data_set.loc[:, data_set.columns.isin(['Credit_Limit', 'Customer_Age', 'Gender', 'Education_Level', 'Income_Category' , 'Months_on_book'])]
settings = data_set.loc[:, data_set.columns.isin(['Customer_Age', 'Gender', 'Education_Level', 'Income_Category' , 'Months_on_book'])]

settings

,Customer_Age,Gender,Education_Level,Income_Category,Months_on_book
CLIENTNUM,,,,,
768805383,45,1,2,3,39
818770008,49,0,3,1,44
713982108,51,1,3,4,36
769911858,40,0,2,1,34
709106358,40,1,1,3,21
...,...,...,...,...,...
772366833,50,1,3,2,40
710638233,41,1,1,2,25
716506083,44,0,2,1,36


1.1 Подключение необходимых библиотек

In [70]:
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import datasets


1.2 Построение классификатора SVM

In [80]:
# Разделение выборки на обучающаю и тестовую
x_train, x_test, y_train, y_test = train_test_split(settings, M_status, test_size=0.35, random_state=2020)

In [ ]:
clf = SGDClassifier(loss="log", penalty="l2")
# y_train__ = y_train.ravel()
clf = clf.fit(x_train, y_train)

In [ ]:
y_pred = clf.predict(x_test)
clf.score(x_test, y_test)

In [ ]:
clf2 = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
clf2.fit(x_train, y_train)

In [ ]:
y_pred = clf2.predict(x_test)
clf2.score(x_test, y_test)

1.3 Оценка точности построенного классификатора с помощью метрик: precision, recall и F1

In [ ]:
from sklearn.model_selection import cross_val_score
print("accuracy:"+str(np.average(cross_val_score(clf, x_test, y_test, scoring='accuracy'))))
print("f1:"+str(np.average(cross_val_score(clf, x_test, y_test, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(clf, x_test, y_test, scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(clf, x_test, y_test, scoring='recall'))))

2.1 Построение классификатора типа "Random Forest" для нашей задачи классификации

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
x_train, x_validation, y_train, y_validation = train_test_split(settings, M_status, test_size=.35, random_state=1)

tree = DecisionTreeClassifier(random_state=241, max_depth=5)  # наилучшая максимальная глубина
tree = tree.fit(x_train, y_train)
# for i in range(3, 30):
#     tree = DecisionTreeClassifier(random_state=241, max_depth = i)
#     tree = tree.fit(x_train, y_train)
#     print(str(i) + ": " + str(tree.score(x_validation, y_validation)))

In [ ]:
from sklearn.metrics import accuracy_score
validation_acc = accuracy_score(y_validation, tree.predict(x_validation))
print('Точность на валидации:', validation_acc)


from sklearn.model_selection import cross_val_score
print("accuracy:"+str(np.average(cross_val_score(tree, x_validation, y_validation, scoring='accuracy'))))
print("f1:"+str(np.average(cross_val_score(tree, x_validation, y_validation, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(tree, x_validation, y_validation, scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(tree, x_validation, y_validation, scoring='recall'))))

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GridSearchCV

param_grid = { 'n_estimators': [200, 300, 400],'max_features': ['auto'], 'max_depth' : [5], 'criterion' :['gini']}

RFC = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv= 5, refit = True) 
RFC.fit(x_train, y_train)


1.3 Оценка точности построенного классификатора с помощью метрик: precision, recall и F1

In [ ]:
print("accuracy:"+str(np.average(cross_val_score(RFC.best_estimator_, x_validation, y_validation, scoring='accuracy'))))
print("f1:"+str(np.average(cross_val_score(RFC.best_estimator_, x_validation, y_validation, scoring='f1'))))
print("precision:"+str(np.average(RFC(tree.best_estimator, x_validation, y_validation, scoring='precision'))))
print("recall:"+str(np.average(RFC(tree.best_estimator, x_validation, y_validation, scoring='recall'))))